In [1]:
#final DASHBOARD

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import base64
from io import BytesIO
from datetime import datetime, time
import pytz
import os
import webbrowser
import logging

# --- CONFIGURATION & UNICODE FONT FIX ---
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

plt.rcParams['font.family'] = [
    'Nirmala UI', 'MS Gothic', 'Segoe UI', 'Arial Unicode MS', 
    'SimSun', 'DejaVu Sans', 'sans-serif'
]
plt.rcParams['axes.unicode_minus'] = False 

ist = pytz.timezone('Asia/Kolkata')
current_time_obj = datetime.now(ist)

def fig_to_base64(fig):
    if fig is None: return None
    tmpfile = BytesIO()
    fig.savefig(tmpfile, format='png', bbox_inches='tight', facecolor='#111111', dpi=150)
    plt.close(fig)
    return base64.b64encode(tmpfile.getvalue()).decode('utf-8')

# --- DATA PREP ---
Apps_Df = pd.read_csv("Play Store Data.csv")
Apps_Df['Installs'] = Apps_Df['Installs'].astype(str).str.replace(r'[+,]', '', regex=True)
Apps_Df['Installs'] = pd.to_numeric(Apps_Df['Installs'], errors='coerce').fillna(0)
Apps_Df['Rating'] = pd.to_numeric(Apps_Df['Rating'], errors='coerce')
Apps_Df['Reviews'] = pd.to_numeric(Apps_Df['Reviews'], errors='coerce').fillna(0)
Apps_Df['Last Updated'] = pd.to_datetime(Apps_Df['Last Updated'], errors='coerce')

# --- TASK 1 COMPONENTS ---
fig1 = None
if 2 <= current_time_obj.hour < 20:
    f_apps = Apps_Df[(Apps_Df['Rating'] >= 4) & (Apps_Df['Last Updated'].dt.month == 1)]
    c_grp = f_apps.groupby('Category').agg({'Rating': 'mean', 'Reviews': 'sum'}).sort_values('Reviews', ascending=False).head(10)
    fig1 = px.bar(c_grp.reset_index().melt(id_vars='Category', value_vars=['Rating', 'Reviews']),
                 x='Category', y='value', color='variable', barmode='group',
                 color_discrete_map={'Rating':'#38bdf8', 'Reviews':'#818cf8'}, template="plotly_dark")
    fig1.update_layout(margin=dict(t=10, b=80), paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

PIE_CHART = px.pie(values=Apps_Df['Type'].value_counts().values, names=Apps_Df['Type'].value_counts().index, 
                   color_discrete_sequence=['#2dd4bf', '#2563eb'], template="plotly_dark")
PIE_CHART.update_layout(margin=dict(t=10, b=10), paper_bgcolor='rgba(0,0,0,0)')

HISTOGRAM = px.histogram(Apps_Df, x='Rating', nbins=20, 
                         color_discrete_sequence=['#a855f7'], template="plotly_dark")
HISTOGRAM.update_layout(margin=dict(t=10, b=40), paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

# --- OTHER TASKS LOGIC ---
def run_t2():
    if not (2 <= current_time_obj.hour < 20): return None
    df = Apps_Df.copy()
    raw_df = pd.read_csv("Play Store Data.csv")
    df['Price'] = pd.to_numeric(raw_df['Price'].astype(str).str.replace('$', '', regex=False), errors='coerce').fillna(0)
    df['Revenue'] = df['Installs'] * df['Price']
    top_cats = df['Category'].value_counts().head(3).index
    res = df[df['Category'].isin(top_cats)].groupby(['Category', 'Type']).agg({'Installs': 'mean', 'Revenue': 'mean'}).reset_index()
    fig, ax1 = plt.subplots(figsize=(10, 6), facecolor='#111111')
    ax1.set_facecolor('#111111')
    labels = [f"{r['Category']}\n({r['Type']})" for _, r in res.iterrows()]
    ax1.bar(np.arange(len(labels)), res['Installs'], 0.5, color='#66fcf1', alpha=0.8, label='Avg Installs')
    ax2 = ax1.twinx()
    ax2.plot(np.arange(len(labels)), res['Revenue'], color='#f0a500', marker='o', linewidth=3, label='Avg Revenue ($)')
    ax1.set_xticks(np.arange(len(labels)))
    ax1.set_xticklabels(labels, color='white', fontsize=9, fontweight='bold')
    ax1.tick_params(colors='white'); ax2.tick_params(colors='white')
    ax1.legend(loc='upper left', facecolor='#111111', labelcolor='white', frameon=False)
    ax2.legend(loc='upper right', facecolor='#111111', labelcolor='white', frameon=False)
    return fig

def run_t3():
    df = Apps_Df.copy()
    df["Country"] = np.random.choice(["India", "United States", "United Kingdom", "Canada", "Germany", "France", "Brazil", "Australia", "Japan", "South Africa"], size=len(df))
    if not (time(0, 0) <= current_time_obj.time() <= time(20, 0)): return None
    df_f = df[~df["Category"].str.startswith(("A", "C", "G", "S"), na=False)]
    top_cat = df_f.groupby("Category")["Installs"].sum().sort_values(ascending=False).head(5).index
    df_map = df_f[df_f["Category"].isin(top_cat)].groupby(["Country", "Category"], as_index=False).agg({"Installs": "sum"})
    iso = {"India": "IND", "United States": "USA", "United Kingdom": "GBR", "Canada": "CAN", "Germany": "DEU", "France": "FRA", "Brazil": "BRA", "Australia": "AUS", "Japan": "JPN", "South Africa": "ZAF"}
    df_map["ISO_Code"] = df_map["Country"].map(iso)
    map_fig = px.choropleth(df_map, locations="ISO_Code", color="Installs", animation_frame="Category", 
                            template="plotly_dark", color_continuous_scale="Viridis")
    map_fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', margin=dict(t=0, b=0, l=0, r=0))
    return map_fig

def run_t4():
    if not (2 <= current_time_obj.hour < 20): return None
    df = Apps_Df.copy()
    df = df[(df['Rating'] >= 4.2) & (df['Category'].str.startswith(('T', 'P'))) & (df['Reviews'] > 1000)]
    df['Month'] = df['Last Updated'].dt.to_period('M').astype(str)
    p_df = df.groupby(['Month', 'Category'])['Installs'].sum().reset_index().pivot(index='Month', columns='Category', values='Installs').fillna(0).cumsum()
    fig, ax = plt.subplots(figsize=(12, 7), facecolor='#111111')
    ax.set_facecolor('#111111')
    p_df.plot(kind="area", stacked=True, ax=ax, alpha=0.8)
    h, l = ax.get_legend_handles_labels()
    trans = {"TRAVEL_AND_LOCAL": "Voyage et Local", "PRODUCTIVITY": "Productividad", "PHOTOGRAPHY": "写真"}
    ax.legend(h, [trans.get(label, label) for label in l], facecolor="#111111", labelcolor='white', frameon=False, prop={'weight': 'bold'})
    ax.tick_params(colors='white')
    ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f"{x/1e9:.1f}B"))
    return fig

def run_t5():
    if not (2 <= current_time_obj.hour < 20): return None
    data = {'App': ['War Zone', 'Makeup Kit', 'Work Hub', 'Love Chat', 'Photo Lab', 'Connect', 'Grid Play'],
            'Category': ['Game', 'Beauty', 'Business', 'Dating', 'Entertainment', 'Social', 'Game'],
            'Rating': [4.5, 4.2, 4.0, 3.8, 4.4, 4.0, 4.8],
            'Size_MB': [45, 20, 15, 30, 50, 35, 60],
            'Installs': [1000000, 200000, 100000, 500000, 300000, 900000, 5000000]}
    df = pd.DataFrame(data)
    df = df[~df['App'].str.contains(r's', case=False, regex=True)]
    trans_map = {'Beauty': 'सौंदर्य (Hindi)', 'Business': 'வணிகம் (Tamil)', 'Game': 'Game'}
    df['Category'] = df['Category'].map(lambda x: trans_map.get(x, x))
    fig, ax = plt.subplots(figsize=(12, 8), facecolor='#111111')
    ax.set_facecolor('#111111')
    for cat in df['Category'].unique():
        sub = df[df['Category'] == cat]
        ax.scatter(sub['Size_MB'], sub['Rating'], s=(sub['Installs']/500)+500, alpha=0.6, edgecolors='white', label=cat)
    ax.legend(loc='upper left', facecolor='#111111', labelcolor='white', frameon=False, prop={'weight': 'bold'})
    ax.tick_params(colors='white')
    return fig

def run_t6():
    if not (time(2, 0) <= current_time_obj.time() <= time(20, 0)): return None
    df = Apps_Df.copy()
    df = df[(df['Reviews'] > 500) & (~df['App'].str.contains('s', case=False)) & (df['Category'].str.startswith(('E', 'C', 'B')))]
    t6_trans = {'BEAUTY': 'सौंदर्य (Hindi)', 'BUSINESS': 'வணிகம் (Tamil)'}
    df['Category'] = df['Category'].map(lambda x: t6_trans.get(x, x))
    df['Month'] = df['Last Updated'].dt.to_period('M').dt.to_timestamp()
    monthly = df.groupby(['Month', 'Category'])['Installs'].sum().reset_index()
    fig, ax = plt.subplots(figsize=(10, 7), facecolor='#111111')
    ax.set_facecolor('#111111')
    colors = ['#00FFFF', '#8F00FF', '#FF69B4']
    for idx, cat in enumerate(monthly['Category'].unique()):
        data = monthly[monthly['Category'] == cat]
        ax.plot(data['Month'], data['Installs'], marker='o', color=colors[idx % 3], label=cat)
    ax.set_yscale('log')
    ax.legend(facecolor='#111111', labelcolor='white', frameon=False, prop={'weight': 'bold'})
    ax.tick_params(colors='white')
    return fig

# --- FINAL DASHBOARD LAYOUT CONFIG ---
task_sections = [
    {
        "banner": "TASK 1 : Category-wise Comparison of Average Rating and Reviews for Top Installed Apps",
        "charts": {
            "User Engagement": (fig1, True),
            "Market Distribution": (PIE_CHART, True),
            "Rating Frequency": (HISTOGRAM, True, True)
        }
    },
    {
        "banner": "Task 2: Free vs Paid Apps",
        "charts": {"Performance Metrics": (fig_to_base64(run_t2()), False, True)}
    },
    {
        "banner": "Task 3: Global Install Distribution by App Category",
        "charts": {"Map View": (run_t3(), True, True)}
    },
    {
        "banner": "Task 4: Cumulative Installs Trend by Category Using Stacked Area Chart",
        "charts": {"Growth Trend": (fig_to_base64(run_t4()), False, True)}
    },
    {
        "banner": "Task 5: App Size vs Rating Analysis Using Bubble Chart",
        "charts": {"Size vs Rating": (fig_to_base64(run_t5()), False, True)}
    },
    {
        "banner": "Task 6: Category-Based Install Growth Trend Over Time with Significant Growth Highlighting",
        "charts": {"Growth Trajectory": (fig_to_base64(run_t6()), False, True)}
    }
]

def render_chart(title, content_tuple):
    content, is_plotly = content_tuple[0], content_tuple[1]
    is_special = len(content_tuple) > 2
    centered_card = "centered-card" if is_special else ""
    
    if content is None:
        inner = '<div class="time-lock"><span>🔒</span><br>Available 02:00 - 12:00 IST</div>'
    elif is_plotly:
        inner = pio.to_html(content, full_html=False, config={'displayModeBar': False})
    else:
        inner = f'<img class="chart-img" src="data:image/png;base64,{content}">'
    return f'<div class="card {centered_card}"><div class="card-header">{title.upper()}</div><div class="chart-content">{inner}</div></div>'

content_html = ""
for section in task_sections:
    content_html += f'<div class="section-banner">{section["banner"]}</div>'
    for k, v in section["charts"].items():
        content_html += render_chart(k, v)

html_dashboard = f"""
<!DOCTYPE html><html><head><meta charset="UTF-8">
<style>
    :root {{ --bg: #0f172a; --card: #1e293b; --accent: #38bdf8; --text: #f1f5f9; }}
    body {{ background: var(--bg); color: var(--text); font-family: 'Nirmala UI', 'Segoe UI', 'MS Gothic', sans-serif; margin: 0; padding: 40px; }}
    header {{ display: flex; align-items: center; justify-content: space-between; max-width: 1400px; margin: 0 auto 40px auto; border-bottom: 2px solid var(--accent); padding-bottom: 20px; }}
    .section-banner {{ grid-column: span 2; background: #0ea5e9; color: white; padding: 15px; border-radius: 8px; font-weight: bold; text-align: center; margin-top: 30px; font-size: 1.2rem; box-shadow: 0 4px 6px -1px rgba(0,0,0,0.3); }}
    .google-wordmark {{ height: 45px; }} .play-icon {{ height: 50px; }}
    .title-group {{ text-align: center; flex-grow: 1; }}
    h1 {{ margin: 0; font-size: 2.1rem; text-transform: uppercase; }}
    .grid {{ display: grid; grid-template-columns: repeat(2, 1fr); gap: 25px; max-width: 1400px; margin: auto; }}
    .card {{ background: var(--card); border-radius: 16px; border: 1px solid #334155; overflow: hidden; }}
    .centered-card {{ width: calc(50% - 12.5px); grid-column: span 2; margin: 0 auto; }}
    .card-header {{ background: rgba(0,0,0,0.2); padding: 15px 20px; font-weight: 700; color: var(--accent); border-bottom: 1px solid #334155; }}
    .chart-content {{ padding: 10px; min-height: 350px; display: flex; align-items: center; justify-content: center; }}
    img.chart-img {{ width: 100%; border-radius: 8px; }}
</style>
</head><body>
    <header>
        <img class="google-wordmark" src="https://upload.wikimedia.org/wikipedia/commons/2/2f/Google_2015_logo.svg">
        <div class="title-group">
            <h1>Google Playstore Analytics Review Dashboard</h1>
            <p style="color: #64748b; margin-top: 10px;">Sync: {current_time_obj.strftime('%Y-%m-%d %H:%M')} IST</p>
        </div>
        <img class="play-icon" src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Play_Arrow_logo.svg">
    </header>
    <div class="grid">
        {content_html}
    </div>
</body></html>
"""

with open("pro_dashboard.html", "w", encoding="utf-8") as f: f.write(html_dashboard)
webbrowser.open('file://' + os.path.abspath("pro_dashboard.html"))


True